In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
from dask_gateway import Gateway

gateway = Gateway()
options = gateway.cluster_options()
options

Options<image='registry.eofactory.ai:5000/dhub/pangeo-vscode:latest',
        worker_cores=2,
        worker_memory=4.0,
        gpu=False>


In [18]:
import numpy as np
import xarray as xr
import stackstac
import rioxarray
import os
from dask.distributed import Client, progress

In [40]:
list_clusters = gateway.list_clusters()
if list_clusters:
    cluster = gateway.connect(list_clusters[0].name)
else:
    cluster = gateway.new_cluster(options)
cluster

In [41]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/dhub.e93396e2bd354054be33a2846a84c814/status,


In [14]:
%%time
import pandas as pd
import geopandas as gpd
import dask_geopandas as dgpd
# _farms = gpd.read_file(f"../aoi/Telangana_4/Telangana_4.shp").to_crs(epsg=32644).copy()
start = 0
size = 100000
_farms = gpd.read_file("../aoi/Telangana/Telangana_26/Telangana_26.fgb", engine='pyogrio', rows = slice(start,start+size))
while True:
    start += size
    # print(start+size)
    ds = gpd.read_file("../aoi/Telangana/Telangana_26/Telangana_26.fgb", engine='pyogrio', rows = slice(start+1,start+size))
    if len(ds) <= 0:
        break
    # _farms = gpd.GeoDataFrame(pd.concat([_farms, ds], ignore_index=True))

100001
200001
300001
400001
500001
600001
700001
800001
900001
1000001
1100001
1200001
1300001
1400001
1500001
1600001
1700001
1800001
1900001
2000001
2100001
2200001
2300001
2400001
2500001
2600001
2700001
2800001
2900001
3000001
3100001
3200001
3300001
CPU times: user 1min 31s, sys: 9.15 s, total: 1min 40s
Wall time: 1min 39s


In [7]:
def search(bbox_4326, date):
    import pystac_client
    api = pystac_client.Client.open("https://stactest.eofactory.ai")
    
    search_2023 = api.search(
        datetime=["2022-12","2023"],
        collections=["sentinel-1-rtc-mosaic-telangana"],
        bbox=bbox_4326,
        max_items=27
    )
    items = search_2023.item_collection()
    return items

def predict_chunk(chunk):
    import pandas as pd
    import dask.array as da 
    tmp=10*np.log10(chunk) #
    tmp=tmp.to_numpy()
    times, height, width = chunk.shape
    print(times, height, width)
    tmp1=tmp.reshape(times, height*width).T
    print(tmp1.shape)
    tmp1[np.isnan(tmp1)] = 0
    # print(tmp1.shape)
    df = pd.DataFrame(data=tmp1)
    df = df.interpolate(method='linear', limit_direction='both', axis=0)
    tmp1 = df.to_numpy()
    print(tmp1.shape)
    result = model.predict(kernel.transform(tmp1))
    chunk_result = chunk.isel(time=0)
    chunk_result.data = result.T.reshape(height, width)
    return chunk_result.astype("uint8")

def predict(items, bbox_4326):
    import stackstac
    # signed_items = [pc.sign(item).to_dict() for item in items]
    # dict_items = [item.to_dict() for item in items]
    data = stackstac.stack(
            items,
            bounds_latlon=bbox_4326,
            dtype='float32',
            assets=["VH"],  # vh only
            chunksize=(27,1,256,256),
            resolution=10,
        ).where(lambda x: x > -32768, other=np.nan)  # sentinel-1-rtc uses -32768 as nodata
        # .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
    
    cliped = data.squeeze(axis=1)
    array = cliped.chunk({'x': 256,'y':256 , 'time':27}) 
    predictor = array.map_blocks(predict_chunk, template= array.isel(time=0).astype("uint8"))
    
    return predictor.compute()

import pickle
model_filekernel = '../training/kernel_poly.sav'
model_filename = '../training/linearsvc_model_poly.sav'

# load the model from disk
kernel= pickle.load(open(model_filekernel, 'rb'))
model = pickle.load(open(model_filename, 'rb'))

model, kernel

(Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('linearsvc',
                  LinearSVC(max_iter=10000, random_state=0, tol=1e-05))]),
 PolynomialCountSketch(degree=3, gamma=1.2, n_components=270, random_state=1))

In [21]:
def create_raster_zone(farms, name):
    from geocube.api.core import make_geocube
    from minio import Minio
    from minio.error import S3Error
    import os
    bucket = "test"
    prefix = "farm_crop_raster"
    out_grid = make_geocube(
        vector_data=farms,
        measurements=["FID"],
        output_crs="EPSG:32644", 
        resolution=(10,-10),
        fill=0
    ).astype('uint32')
             
    out_grid["FID"].rio.to_raster(f"../farm_crop_raster/{name}", driver='COG')
    client = Minio(
            "data.skymapglobal.vn",
            access_key='P3OWYdq0s6CrYSS4zJsZ',
            secret_key='K3vcj0NOm8hNnfJuZxXeYs9hAO0jHy80buZCourE'
        )
    client.fput_object(bucket, f"{prefix}/{name}", f"../farm_crop_raster/{name}")
    # os.remove(f"./farm_crop_raster/{name}")
    return f"https://data.skymapglobal.vn/{bucket}/{prefix}/{name}"

In [22]:
import math
def genarate_grid(bbox, size):
    x = math.ceil((bbox[2] - bbox[0]) / size)
    y = math.ceil((bbox[3] - bbox[1]) / size)
    x, y 
    box_grids = []
    for i in range(x):
        for j in range(y):
            box_grids.append([bbox[0]+ size*i, bbox[1]+ size*j, bbox[0]+ size*(i+1), bbox[1]+ size*(j+1)])
    return box_grids

In [42]:
def classify(grid_farms, crop_zone_url):
    import rasterio
    from rasterio.warp import transform_bounds
    from xrspatial.zonal import stats
    import pandas as pd
    import rioxarray
    chunk_size = 256
    final_ds = 1
    index = 0
    print("Total farms:", grid_farms.shape[0])
    if grid_farms.shape[0] > 0 :
        bbox = grid_farms.total_bounds
        bbox_4326 = np.array(transform_bounds(32644, 4326, *bbox))
        done = False
        # while not done:
        #     try:
        crop_zone_vrt = rasterio.vrt.WarpedVRT(rasterio.open(crop_zone_url)
                                               # , transform=predict_mask.rio.transform(), height=predict_mask.shape[0], width=predict_mask.shape[1]
                                              )
        crop_zone_ds = rioxarray.open_rasterio(crop_zone_vrt, chunks=(1, chunk_size, chunk_size)).squeeze(axis=0).astype("uint32")
        
        predict_vrt = rasterio.vrt.WarpedVRT(rasterio.open('/vsicurl/https://data.skymapglobal.vn/sentinel-1-rtc-mosaic-masks/mosaicked_mask_karnataka.tif'),
                                            transform=crop_zone_ds.rio.transform(), height=crop_zone_ds.shape[0], width=crop_zone_ds.shape[1])
        predict_mask = rioxarray.open_rasterio(predict_vrt, chunks=(1, chunk_size, chunk_size)).squeeze(axis=0).astype("uint32")
        predict_ds = predict_mask.chunk(chunk_size,chunk_size)
        print('CROP SHAPE:', crop_zone_ds.shape)
        print('PREDICT SHAPE:', predict_ds.shape)
        dask_stats_df = stats(zones=crop_zone_ds, values=predict_ds)
        
        print("Zonal statistic...")
        result = dask_stats_df.compute()
        done = True
            # except Exception as e:
            #     print(e)
            #     print('Zonal failed! Runing again...')
        if isinstance(final_ds, int):
            final_ds = result.copy()
        else:
            final_ds = pd.concat([final_ds, result.copy()])

    return final_ds

In [23]:
def drop_duplicates(df):
    df = dgpd.from_geopandas(df, npartitions=100)
    result = df.drop_duplicates(subset=['FID'], ignore_index=True).compute()
    return result

In [45]:
import glob
files = glob.glob('../farm_by_district/Karnataka/*')
files

['../farm_by_district/Karnataka/BIDAR.fgb',
 '../farm_by_district/Karnataka/RAICHUR.fgb',
 '../farm_by_district/Karnataka/DAKSHINA_KANNADA.fgb',
 '../farm_by_district/Karnataka/UTTARA_KANNADA.fgb',
 '../farm_by_district/Karnataka/VIJAYAPURA.fgb',
 '../farm_by_district/Karnataka/DAVANGERE.fgb',
 '../farm_by_district/Karnataka/BAGALKOTE.fgb',
 '../farm_by_district/Karnataka/BELAGAVI.fgb',
 '../farm_by_district/Karnataka/KALABURAGI.fgb',
 '../farm_by_district/Karnataka/CHIKKAMAGALURU.fgb',
 '../farm_by_district/Karnataka/CHIKKABALLAPURA.fgb',
 '../farm_by_district/Karnataka/MYSURU.fgb',
 '../farm_by_district/Karnataka/KODAGU.fgb',
 '../farm_by_district/Karnataka/HAVERI.fgb',
 '../farm_by_district/Karnataka/BENGALURU_URBAN.fgb',
 '../farm_by_district/Karnataka/HASSAN.fgb',
 '../farm_by_district/Karnataka/CHITRADURGA.fgb',
 '../farm_by_district/Karnataka/RAMANAGARA.fgb',
 '../farm_by_district/Karnataka/KOPPAL.fgb',
 '../farm_by_district/Karnataka/YADGIR.fgb',
 '../farm_by_district/Karnataka

In [46]:
classified = glob.glob('../results/Karnataka/*')
classified

['../results/Karnataka/CLASSIFIED_FARMS_UTTARA_KANNADA.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_CHAMARAJANAGARA.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_BALLARI.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_KOLAR.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_YADGIR.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_VIJAYAPURA.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_DAVANGERE.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_CHITRADURGA.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_MANDYA.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_UDUPI.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_KALABURAGI.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_DAKSHINA_KANNADA.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_BIDAR.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_BELAGAVI.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_HASSAN.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_BENGALURU_RURAL.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_RAMANAGARA.fgb',
 '../results/Karnataka/CLASSIFIED_FARMS_KOD

In [49]:
# farms2 = farms.buffer(2).buffer(-2)
# def clean_farms(index):
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as dgpd
size = 100000
slice = 100000
for f in files:
    print(f)
    file_name = os.path.basename(f).split('.')[0]
    if f'{file_name}.fgb' in [f"{'_'.join(f.split('_')[2:])}" for f in classified]:
        print('Done! Zonal next image ... ')
        continue
    print(file_name)
    _farm = gpd.read_file(f, engine='pyogrio')
    # farm = dgpd.from_geopandas(_farm, npartitions=100)
    # bbox = _farm.total_bounds
    farm = _farm[['FID', 'geometry']]
    print(farm.shape)
    
    start = -1
    slice_idx = 0
    while True:
        print(slice_idx)
        slice_farm = dgpd.from_geopandas(farm.iloc[start+1:start+slice+1, :], npartitions=100).compute()
        if len(slice_farm) <= 0:
            break
        bbox = slice_farm.total_bounds
        box_grids = genarate_grid(bbox.tolist(), size)
        
        for idx, grid in enumerate(box_grids):
            print('Grid', idx)
            farms_grid = dgpd.from_geopandas(slice_farm, npartitions=100)
            farms_grid = farms_grid.clip(mask=grid).compute()
            if len(farms_grid) <= 0: 
                continue
                
            crop_zone_url = create_raster_zone(farms_grid, f'KARNATAKA_{file_name}_{slice_idx}_{idx}.tif')
            classified_farms = classify(farms_grid, crop_zone_url)
            if not isinstance(classified_farms, int):
                farms = farms_grid.merge(classified_farms, how='left', left_on='FID', right_on='zone')
                if idx == 0:
                    final_farms = farms
                else:
                    final_farms = gpd.GeoDataFrame(pd.concat([final_farms, farms], ignore_index=True))
                    final_farms = drop_duplicates(final_farms)
    
        if start == -1:
            merge_slice_farms = final_farms
        else:
            merge_slice_farms = gpd.GeoDataFrame(pd.concat([merge_slice_farms, final_farms], ignore_index=True))
    
        start += slice
        slice_idx += 1
    
    print('Writing to file ...')
    merge_slice_farms.to_file(f"../results/Karnataka/CLASSIFIED_FARMS_{file_name}.fgb", driver='FlatGeobuf', engine='pyogrio')

../farm_by_district/Karnataka/BIDAR.fgb
Done! Zonal next image ... 
../farm_by_district/Karnataka/RAICHUR.fgb
Done! Zonal next image ... 
../farm_by_district/Karnataka/DAKSHINA_KANNADA.fgb
Done! Zonal next image ... 
../farm_by_district/Karnataka/UTTARA_KANNADA.fgb
Done! Zonal next image ... 
../farm_by_district/Karnataka/VIJAYAPURA.fgb
Done! Zonal next image ... 
../farm_by_district/Karnataka/DAVANGERE.fgb
Done! Zonal next image ... 
../farm_by_district/Karnataka/BAGALKOTE.fgb
BAGALKOTE
(1112917, 2)
0
Grid 0
Total farms: 100000
CROP SHAPE: (2689, 7237)
PREDICT SHAPE: (2689, 7237)
Zonal statistic...
1
Grid 0
Total farms: 100000
CROP SHAPE: (4082, 3659)
PREDICT SHAPE: (4082, 3659)
Zonal statistic...
2
Grid 0
Total farms: 100000
CROP SHAPE: (4130, 8071)
PREDICT SHAPE: (4130, 8071)
Zonal statistic...
3
Grid 0
Total farms: 100000
CROP SHAPE: (2176, 3653)
PREDICT SHAPE: (2176, 3653)
Zonal statistic...
4
Grid 0
Total farms: 100000
CROP SHAPE: (4060, 3544)
PREDICT SHAPE: (4060, 3544)
Zonal st

In [48]:
del merge_slice_farms, final_farms